In [1]:
!pip install langchain-core langgraph>0.2.27

In [6]:
from google.colab import userdata

NVIDIA_API_KEY = userdata.get('NVIDIA_API_KEY')
LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')
LANGCHAIN_TRACING_V2 = userdata.get('LANGCHAIN_TRACING_V2')
LANGCHAIN_PROJECT = userdata.get('LANGCHAIN_PROJECT')

print(NVIDIA_API_KEY)
print(LANGCHAIN_API_KEY)
print(LANGCHAIN_TRACING_V2)
print(LANGCHAIN_PROJECT)

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
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
true
project2


In [9]:
!pip install -qU langchain-nvidia-ai-endpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 45.8 MB/s eta 0:00:00


In [13]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

model = ChatNVIDIA(model="meta/llama3-70b-instruct", api_key=NVIDIA_API_KEY)

In [14]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'role': 'assistant', 'content': "Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", 'token_usage': {'prompt_tokens': 15, 'total_tokens': 41, 'completion_tokens': 26}, 'finish_reason': 'stop', 'model_name': 'meta/llama3-70b-instruct'}, id='run-bcac3beb-7f27-4b2a-a5f5-55998b3024e2-0', usage_metadata={'input_tokens': 15, 'output_tokens': 26, 'total_tokens': 41}, role='assistant')

In [19]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='I remember! Your name is Bob!', additional_kwargs={}, response_metadata={'role': 'assistant', 'content': 'I remember! Your name is Bob!', 'token_usage': {'prompt_tokens': 40, 'total_tokens': 49, 'completion_tokens': 9}, 'finish_reason': 'stop', 'model_name': 'meta/llama3-70b-instruct'}, id='run-1745c801-5e5b-40a6-aa7a-68b8b4e4cae0-0', usage_metadata={'input_tokens': 40, 'output_tokens': 9, 'total_tokens': 49}, role='assistant')

In [52]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

from langchain_core.messages import SystemMessage, trim_messages

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm aarij"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [57]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "English"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Bob! It's nice to meet you. I'm happy to assist you with any questions or topics you'd like to discuss. What's on your mind today?


In [58]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked "what's 2 + 2?" and the answer is 4!


In [64]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="")

Hi Todd! Here's a joke for you:

Why couldn't the bicycle stand up by itself?

(Wait for it...)

Because it was two-tired!

Hope that brought a smile to your face, Todd!